In [1]:
import pandas as pd 
import numpy as np
#importing libraries
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import math

In [2]:
def merge_left_right(data1,data2):
    print(len(data1),len(data2))
    temp_data = data1.append(data2,ignore_index=True)
    return temp_data

def merg_sort(list_data):
    list_length = len(list_data)
    if list_length == 1:
        output = list_data[0][2]
        return output
    else:
        mid_point = math.floor(list_length/2)
        #print(len(list_data[:mid_point]))
        left = list_data[mid_point:]
        right = list_data[:mid_point]
        return merge_left_right(merg_sort(left), merg_sort(right))

In [3]:
data = pd.read_csv('Security_and_market_movement_unscaled_cluster_2019-02-28.csv')

In [4]:
data.drop(columns=['Unnamed: 0'],inplace=True)

In [5]:
data.head()

,SecurityID,KeyDate,YieldWorst,ModifiedDuration_Plain,AmtOutstanding,Target_G_change,Group,G_change,skew_change,swap_change,vix_change,Close_change,slope_change,skew_change_Weekly,swap_change_Weekly,vix_change_Weekly,Close_change_Weekly,slope_change_Weekly
0,56535,2019-01-02,26.9747,1.85247,300000000.0,0.002132,Cluster 10,0.014612,-0.004584,-0.004966,-0.086546,0.001269,-0.238095,0.000000,0.0,0.00000,0.000000,-0.026617
1,56535,2019-01-03,26.7432,1.84797,300000000.0,-0.005914,Cluster 10,0.007046,-0.042981,0.000000,0.096038,-0.024757,0.062500,0.000000,0.0,0.00000,0.000000,-0.024643
2,56535,2019-01-04,26.9556,1.84352,300000000.0,0.006400,Cluster 10,0.017257,-0.006951,-0.012668,-0.159921,0.034336,0.000000,0.000000,0.0,0.00000,0.000000,-0.034643
3,56535,2019-01-07,27.0412,1.84036,300000000.0,-0.001467,Cluster 10,-0.045289,0.019024,0.005443,0.000935,0.007010,0.000000,0.000000,0.0,0.00000,0.000000,-0.025119
4,56535,2019-01-08,26.9041,1.83947,300000000.0,-0.007478,Cluster 10,-0.005836,0.015322,0.020495,-0.043458,0.009695,-0.117647,-0.004034,0.0,-0.03859,0.005511,-0.058648


In [6]:
data_1 = data.drop(columns = ['skew_change_Weekly','swap_change_Weekly','vix_change_Weekly','Close_change_Weekly','slope_change_Weekly','AmtOutstanding'])
data_1 = data_1[data_1['Group'] == 'Cluster 0']
data_1.drop(columns = ['Group'],inplace=True)

In [9]:
new = data_1[['SecurityID','KeyDate','YieldWorst','ModifiedDuration_Plain','Target_G_change','G_change']]

In [11]:
dummy_variable = pd.read_csv('Security_info_post_dummy.csv')

In [12]:
dummy_variable.drop(columns=['Unnamed: 0'],inplace=True)

In [13]:
dummy_variable.head()

,SecurityID,1.5 Lien Secured,1st Lien Secured,1st lien,2nd Lien Secured,3rd Lien Secured,Asset Backed,Jr Subordinated Unsecured,Secured,Sr Unsecured,...,FLOATING,FUNGED,Fixed,PAY-IN-KIND,STEP CPN,VARIABLE,WHEN ISSUED,ZERO COUPON,MinIncrement,MinPiece
0,62,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1000.0,50000.0
1,64,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1000.0,2000.0
2,69,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,100000.0,100000.0
3,72,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000.0,2000.0
4,78,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1000.0,150000.0


In [15]:
new = new.merge(dummy_variable,on=['SecurityID'],how='left')

In [20]:
new.columns

Index(['SecurityID', 'KeyDate', 'YieldWorst', 'ModifiedDuration_Plain',
       'Target_G_change', 'G_change', '1.5 Lien Secured', '1st Lien Secured',
       '1st lien', '2nd Lien Secured',
       ...
       'FLOATING', 'FUNGED', 'Fixed', 'PAY-IN-KIND', 'STEP CPN', 'VARIABLE',
       'WHEN ISSUED', 'ZERO COUPON', 'MinIncrement', 'MinPiece'],
      dtype='object', length=147)

In [18]:
# 1 is leader, 0 is lagger
def lead_lag(x,y):
    if x > y:
        return 1
    else:
        return 0

In [22]:
new['leader/lagger'] = new.apply(lambda x: lead_lag(x.Target_G_change,x.G_change),axis=1)

In [26]:
new.drop(columns=['Target_G_change','G_change'],inplace = True)

### Feature engineering construct the Change

In [28]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [37]:
# split data into X and y
X = new[:,'YieldWorst':'MinPiece']
Y = new['leader/lagger']

TypeError: '(slice(None, None, None), slice('YieldWorst', 'MinPiece', None))' is an invalid key

In [35]:
new

,SecurityID,KeyDate,YieldWorst,ModifiedDuration_Plain,1.5 Lien Secured,1st Lien Secured,1st lien,2nd Lien Secured,3rd Lien Secured,Asset Backed,...,FUNGED,Fixed,PAY-IN-KIND,STEP CPN,VARIABLE,WHEN ISSUED,ZERO COUPON,MinIncrement,MinPiece,leader/lagger
0,56288,2019-01-02,7.05424,6.64285,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1000.0,2000.0,1
1,56288,2019-01-03,6.87398,6.65456,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1000.0,2000.0,0
2,56288,2019-01-04,6.47403,6.69407,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1000.0,2000.0,0
3,56288,2019-01-07,6.28284,6.71163,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1000.0,2000.0,0
4,56288,2019-01-08,6.07889,6.73048,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1000.0,2000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38427,109,2019-02-22,4.19661,6.62624,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1000.0,2000.0,0
38428,109,2019-02-25,4.20100,6.62331,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1000.0,2000.0,1
38429,109,2019-02-26,4.16937,6.62331,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1000.0,2000.0,0
38430,109,2019-02-27,4.22929,6.72373,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1000.0,2000.0,1
